# Генерация НОВЫХ Покемонов используя GPT-2

Этот блокнот позволяет нам создавать изображения Покемонов с помощью модели, которую мы обучили с помощью предыдущего блокнота.

Чтобы использовать этот блокнот, просто следуйте инструкциям и запускайте каждую ячейку в соответствии с указаниями.


## Установка зависимостей

Импортируем всё, что нам требуется

In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files, drive

import os
import sys
import time
import re
import gpt_2_simple as gpt2
import shutil
import math
from PIL import Image

## Подключаем Google Drive

Google Диск используется для загрузки нашей модели и хранения сгенерированных изображений Покемонов. Здесь мы монтируем ваш Google Диск

In [ ]:
drive.mount('/content/drive')

## Импортируем нашу модель


In [ ]:
checkpoint_folder = 'pokemon-gpt-2-checkpoints'
run_name = 'pokemon-gpt-2-run'

if not os.path.exists(f'checkpoint/{run_name}'):
    shutil.copytree(f'/content/drive/My Drive/{checkpoint_folder}/{run_name}', f'checkpoint/{run_name}')

## Генерация

Здесь мы сгенерируем Покемонов с моделью, которую мы импортировали, и сохраним их на Google Диск.

Для этой ячейки можно изменить несколько параметров:

- `output_folder` - Это папка, в которой Покемоны будут сохраняться на вашем Google Диске.
- `generate_count` - Кол-во новых Покемонов, которое будет сгенерировано при запуске этой ячейки.
- `temperature` - Контрольная величина, насколько "сумасшедшим" будет результат. Большое число приведет к чему-то, что вряд ли будет похоже на ввод, но может быть интересным. Слишком низкое значение будет скучным.
- `width` - Ширина выходных изображений.
- `height` - Высота выходных изображений.


In [ ]:
output_folder = 'pokemon-gpt-2-output'
generate_count = 20 #@param {type:"integer"}
temperature = 0.78 #@param {type:"slider", min:0.7, max:1.3, step:0.01}
width = 64
height = 64

import tensorflow as tf

def createImage(text, path):
    lines = text.split('\n')

    imageWidth = 0
    imageHeight = 0

    for line in lines:
        split = line.split(' ')

        marker = split[0]
        match = re.search("^([0-9]+)([du])$", marker)

        if match:
            groups = match.groups()
            index = int(groups[0])

            width = len(split) - 1
            height = index + 1

            if width > imageWidth:
                imageWidth = width;

            if height > imageHeight:
                imageHeight = height

    pixels = []
    for y in range(imageHeight):
        for x in range(imageWidth):
            pixels.append((0, 0, 0, 0))

    for line in lines:
        split = line.split(' ')

        marker = split[0]
        match = re.search("^([0-9]+)([du])$", marker)

        if match:
            groups = match.groups()
            index = int(groups[0])

            for x in range(len(split) - 1):
                s = split[x + 1]

                if s != '~':
                    r = 0
                    g = 0
                    b = 0

                    if s == 'a':
                        r = g = b = 107
                    elif s == 'b':
                        r = g = b = 187
                    else:
                        c = ord(s[0]) - 33

                        b = (c & 3) * 64
                        if b == 192:
                            b += 63

                        c = c >> 2
                        g = (c & 3) * 64
                        if g == 192:
                            g += 63

                        c = c >> 2
                        r = (c & 3) * 64
                        if r == 192:
                            r += 63

                    i = (index * imageWidth) + x

                    pixels[i] = (r, g, b, 255)

    image = Image.new('RGBA', (imageWidth, imageHeight))
    image.putdata(pixels)
    image.save(path)

def blankLines():
    lines = []

    for i in range(0, height):
        lines.append('')

    return lines

sess = None

for ii in range(0, generate_count):
    print(ii)
    tf.reset_default_graph()


    if not sess:
        sess = gpt2.start_tf_sess()
    else:
        sess = gpt2.reset_session(sess)

    gpt2.load_gpt2(sess, run_name=run_name)

    lines = blankLines()
    prefix = ''
    hasColor = False

    while True:
        text = gpt2.generate(sess, run_name=run_name, prefix=prefix, temperature=temperature, return_as_list=True)[0]

        print('\n\nВывод:');
        print(text)

        newLines = text.split('\n')

        direction = None
        lastIndex = None
        for line in newLines:
                split = line.split(' ')[:width + 2]

                if len(split) < 55:
                    break;

                marker = split[0]
                match = re.search("^([0-9]+)([du])$", marker)

                if match:
                    groups = match.groups()

                    try:
                        index = int(groups[0])
                    except:
                        break

                    if direction == None:
                        direction = groups[1]

                    if groups[1] != direction:
                        print('Изменения направления')
                        break

                    if lastIndex != None:
                        if groups[1] == 'd' and index <= lastIndex:
                            break
                        elif groups[1] == 'u' and index >= lastIndex:
                            break
                    lastIndex = index

                    split[0] = marker.replace('u', 'd')

                    if not hasColor:
                        for character in split[2:]:
                            if character != '~' and character != '`':
                                hasColor = True
                                break

                    while len(split) < width:
                        split.append('~')

                    try:
                        lines[index] = ' '.join(split)
                    except IndexError:
                        break

        if not hasColor:
            lines = blankLines()
            continue

        topIndex = None
        for i in range(0, height):
            if lines[i]:
                topIndex = i
                break

        bottomIndex = None
        for i in range(topIndex, height):
            if lines[i]:
                bottomIndex = i
            else:
                break

        sectionSize = 5
        if topIndex > 0:
            section = lines[topIndex:min(topIndex+sectionSize+1, bottomIndex+1)]
            section.reverse()
            for i in range(0, len(section)):
                section[i] = section[i].replace('d', 'u')

        elif bottomIndex < height - 1:
            section = lines[max(bottomIndex-sectionSize, topIndex):bottomIndex+1]

        else:
            print('\n'.join(lines))
            filename = f'{int(time.time())}'
            text = '\n'.join(lines)

            if not os.path.exists(f'/content/drive/My Drive/{output_folder}'):
                os.makedirs(f'/content/drive/My Drive/{output_folder}')


            createImage(text, f'/content/drive/My Drive/{output_folder}/{filename}.png')

            print('Сохранено!')
            break

        prefix = '\n'.join(section)
        print(f'\n\nПрефикс:\n{prefix}')